# Retrieval-Augmented Generation (RAG)




### Introduction

In the world of natural language processing (NLP), models like ChatGPT have become household names. These models are pre-trained on vast amounts of text data up to a certain point in time, known as their "knowledge cutoff." While incredibly versatile, their static nature means they can't incorporate information or events that occur after this cutoff. This is where Retrieval-Augmented Generation (RAG) comes in, blending the generative capabilities of models like ChatGPT with the dynamic, up-to-date knowledge from external sources.


### How RAG Works




![RAG](https://taesiri.xyz/data/rag2.png)



RAG enhances traditional language models through a two-stage process:

1. **Retrieval Stage**: The system queries a continuously updated database or knowledge base to find information relevant to the input query. This allows the model to access the most current data, even if it's beyond its original training cutoff.

2. **Generation Stage**: Leveraging a generative model (e.g., GPT), RAG integrates the context from the retrieved documents to produce informed and relevant text. This step ensures that the generation is not only based on the model's pre-trained knowledge but is also augmented with the latest information.

### Key Applications

- **Question Answering**: RAG systems can answer questions with the most current information, overcoming the knowledge cutoff limitation of standalone generative models.
- **Conversational Agents**: Chatbots powered by RAG can provide users with up-to-date answers, making them more useful for current events and news-related queries.

### Advantages

- **Current Information**: RAG allows language models to break free from their knowledge cutoff, making them more relevant for today's rapidly changing world.
- **Depth and Accuracy**: The retrieval component ensures that the generated content is not only contextually relevant but also deeply informative and factually accurate.
- **Adaptability**: By changing the external data sources, RAG can be tailored to different domains and information needs.

### [Llamaindex](https://www.llamaindex.ai/)

LlamaIndex is a versatile data framework designed to connect custom data sources with large language models (LLMs) like GPT-4. It serves as a bridge between enterprise data and LLM applications, enabling the ingestion, structuring, retrieval, and integration of data for various applications. LlamaIndex allows for the loading of data from over 160 sources in different formats, indexing this data for diverse use cases, and orchestrating LLM workflows efficiently. It offers a comprehensive suite of modules to evaluate LLM application performance and seamlessly integrates with observability partners. Additionally, LlamaIndex boasts a thriving developer network, community contributions, and integration options with various services.


### Project

In this notebook, we'll guide you through the process of leveraging LlamaIndex to enhance information retrieval and text generation. Specifically, we will demonstrate how to use LlamaIndex to upload a PDF document and dissect it into manageable segments. These segments will then be systematically stored in a vector database, designed for efficient querying. When a query is submitted, our system will search this database to find the most relevant document segments. The most pertinent segment – or `chunk` – will be retrieved as the context to address the query. Subsequently, this context will be provided to a compact language model, in this case, [Microsoft Phi-2](https://www.microsoft.com/en-us/research/blog/phi-2-the-surprising-power-of-small-language-models/). We'll then instruct Phi-2 to craft a response, drawing upon both the specific question posed and the context supplied by the selected document chunk. This method showcases the synergy between advanced retrieval techniques and modern language models to generate informed, contextually relevant responses.










## Installing dependencies

(This step might take longer than 10 minutes)

In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q llama-index
!pip install -q gradio
!pip install -q einops
!pip install -q accelerate
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-embeddings-instructor
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-llms-openai

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts import PromptTemplate
import torch

In [ ]:
!mkdir Data

In [ ]:
# TODO - Upload a single PDF document into Data folder

# we will use SimpleDirectoryReader to load all the documents in a folder
documents = SimpleDirectoryReader("./Data").load_data()

In [ ]:
len(documents)

13

In [ ]:
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")
query_wrapper_prompt

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='<|USER|>{query_str}<|ASSISTANT|>')

In [ ]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="microsoft/phi-2",
    model_name="microsoft/phi-2",
    device_map="cuda",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.bfloat16}
)


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en-v1.5 from huggingface for embedding - https://huggingface.co/BAAI/bge-small-en-v1.5

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
# Create an vector database from document chunks

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

# Query the database and return the most relevant conent to the query
def predict(input, history):
  response = query_engine.query(input)
  return str(response)


In [ ]:
# TODO: Try querying the engine with multiple question and examine the response and source_nodes
r = query_engine.query("YOUR QUESTION HERE")

In [ ]:
# examine the response
r

In [ ]:
# examine the source nodes used for the answer
r.source_nodes[0]

### Using a Chat Interface

Below, we have created a chat interface that allows you to ask various questions based on the document stored in it. Please use this chat application to ask 10 different questions and then report your understanding in the cell below.

In [ ]:
import gradio as gr

gr.ChatInterface(predict).launch(share=True)

In [ ]:
# Write your answer and analysis here.